In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sc.constants as cs
from sc import payoff as po
from yq.scripts import simulation as yqsm

print(pd.__version__)

In [ ]:
paths_arr = yqsm.read_sim_data('heston', '20231114_025704_252_0.5', pd.Timestamp('2023-08-14'), pd.Timestamp('2023-08-15'))
print(paths_arr)

In [ ]:
df_plot = paths_arr[0][5].copy(deep = True)
print(po.payouts(df_plot, barrierHit = False))
for asset in cs.ASSET_NAMES:
    df_plot[asset] = df_plot[asset] / cs.INITIAL_LEVELS[asset]
plt.figure(figsize = (15, 8))
plt.plot(df_plot)
plt.axhline(y=1, c = "black", alpha = 0.5)
plt.axhline(y = 0.6, c = "red", alpha = 0.5)
plt.axvline(x = cs.FINAL_FIXING_DATE, c = "darkgreen")
for date in cs.EARLY_REDEMPTION_OBSERVATION_DATES:
    plt.axvline(x = date, c = "green", alpha = 0.5)



In [ ]:
#Entire process

first_date = pd.Timestamp('2023-08-09')
last_date = pd.Timestamp('2023-11-09')

#load simulated paths from csvs
total_data = yqsm.read_sim_data('gbm', '20231114_031302_63', first_date, last_date)

est_price = []
for single_day in total_data:
    #rename columns
    df_sim_array = []
    for df in single_day:
        df_sim_array.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))
    
    prices = po.pricing_multiple(df_sim_array)
    est_price.append(sum(prices) / len(prices))
print(est_price)




In [ ]:
df_product = po.get_product_price(pd.Timestamp('2023-11-09'))
df_product = df_product.loc[first_date : last_date]
df_product['Est_Price'] = est_price

plt.plot(df_product)

In [ ]:
first_date = pd.Timestamp('2023-08-09')
last_date = pd.Timestamp('2023-11-09')

total_data = yqsm.read_sim_data('gbm', '20231114_031302_63', first_date, last_date)

#df_sim = total_data[0][0][0].rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'})
#df_sim_arr = total_data[0][0]


In [ ]:
# entire pricing window

prices = []
deltas = {'LONN.SE': [], 'SIKA.SE': []}
gammas = {'LONN.SE': [], 'SIKA.SE': []}
for single_day in total_data[0]:
    #rename columns
    df_sim_array = []
    for df in single_day:
        df_sim_array.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))
    
    results = po.pricing_with_greeks_multiple(df_sim_array, 0.001)
    price_sum = 0
    greeks_sum = {'LONN.SE': [0, 0], 'SIKA.SE': [0, 0]}
    for sim in results:
        price_sum += sim[0]
        for asset in cs.ASSET_NAMES:
            greeks_sum[asset][0] += sim[1][asset][0]
            greeks_sum[asset][1] += sim[1][asset][1]
    est_price = price_sum / len(results)
    for asset in cs.ASSET_NAMES:
        deltas[asset].append(greeks_sum[asset][0] / len(results))
        gammas[asset].append(greeks_sum[asset][1] / len(results))
    prices.append(est_price)


In [ ]:
print(prices)
df_product = po.get_product_price(pd.Timestamp('2023-11-09'))
df_product = df_product.loc[first_date : last_date]
df_product['Est_Price'] = prices

plt.plot(df_product)

In [ ]:
df_deltas = pd.DataFrame(index = df_product.index, data = deltas)

plt.figure(figsize = [10, 6])
plt.axvline(x = pd.Timestamp('2023-10-23'), color = 'red')
plt.axhline(y = 0, color = 'black')
plt.axvline(x = pd.Timestamp('2023-09-18'), color = 'pink')
plt.plot(df_deltas)

In [ ]:
df_gammas = pd.DataFrame(index = df_product.index, data = gammas)

plt.figure(figsize = [10, 6])
plt.axvline(x = pd.Timestamp('2023-10-23'), color = 'red')
plt.axhline(y = 0, color = 'black')
plt.axvline(x = pd.Timestamp('2023-09-18'), color = 'pink')
plt.plot(df_gammas)

In [ ]:
df_historical = po.get_historical_assets(cs.FINAL_FIXING_DATE, first_date)
today = df_historical.index[-1]
barrierHit = po.checkBarrier(df_historical)

for asset in cs.ASSET_NAMES:
    df_historical[asset] = df_historical[asset] / cs.INITIAL_LEVELS[asset]

plt.figure(figsize = [10, 6])
plt.axhline(y = 1)
plt.axhline(y = 0.6)
plt.plot(df_historical)

In [ ]:
first_date = pd.Timestamp('2023-08-09')
last_date = pd.Timestamp('2023-08-12')

total_data = yqsm.read_sim_data('gbm', '20231111_195045_022812', first_date, last_date)

df_sim = total_data[0][0][0].rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'})
df_sim_arr = total_data[0][0]

df_constant = pd.DataFrame(index = df_sim.index)

In [ ]:
#print(total_data[0][0][0])

#df_sim = total_data[0][0][0].rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'})
#df_sim_arr = total_data[0][0]

first_sim_date = df_sim.first_valid_index()
df_historical = po.get_historical_assets(first_sim_date, cs.INITIAL_FIXING_DATE)
today = df_historical.index[-1]
barrierHit = po.checkBarrier(df_historical)

print(barrierHit)

df_constant = pd.DataFrame(index = df_sim.index, data = {'LONN.SE': [cs.INITIAL_LEVELS['LONN.SE'] * 0.95] * len(df_sim), 'SIKA.SE': [cs.INITIAL_LEVELS['SIKA.SE'] * 0.95] * len(df_sim)})
#print(df_constant)

payouts_arr = po.payouts_h(df_constant, barrierHit, 0.01, 'LONN.SE')
print(payouts_arr)

In [ ]:
rnv_arr = po.rnv_multiple(payouts_arr, today)
print(rnv_arr)
print(po.delta(rnv_arr, 0.01))
print(po.gamma(rnv_arr, 0.01))

In [ ]:
#product price upper bound

df_constant = pd.DataFrame(index = df_sim.index, data = {'LONN.SE': [cs.INITIAL_LEVELS['LONN.SE'] * 0.95] * len(df_sim), 'SIKA.SE': [cs.INITIAL_LEVELS['SIKA.SE'] * 0.95] * len(df_sim)})
#print(df_constant)
#display(df_new)
#print(po.payouts(df_new, False))
dict_upper_bound = {}
today = pd.Timestamp('2023-08-09')
for i in range(67):
    df_payouts = po.payouts(df_constant, False)
    dict_upper_bound[today] = po.rnv_single(df_payouts, today)
    today = df_constant.first_valid_index()
    df_constant = df_constant.tail(len(df_constant) - 1)
df_upper_bound = pd.DataFrame.from_dict(dict_upper_bound, orient = 'index')
df_upper_bound.index.name = 'Date'
df_upper_bound.columns = ['Price']
print(df_upper_bound)


In [73]:
# no autocall

def pricing_multiple_no_autocall(df_sim_array):
    first_sim_date = df_sim_array[0].first_valid_index()
    df_historical = po.get_historical_assets(first_sim_date, cs.INITIAL_FIXING_DATE)
    today = df_historical.index[-1]
    barrierHit = po.checkBarrier(df_historical)
    df_payouts_arr = []
    for df_sim in df_sim_array:
        df_payouts = po.payouts_no_autocall(df_sim, barrierHit)
        df_payouts_arr.append(df_payouts)
    rnv_arr = po.rnv_multiple(df_payouts_arr, today)
    return rnv_arr

first_date = pd.Timestamp('2023-08-09')
last_date = pd.Timestamp('2023-11-09')

#load simulated paths from csvs
total_data = yqsm.read_sim_data('gbm', '20231114_031302_63', first_date, last_date)[0]

est_price = []
for single_day in total_data:
    #rename columns
    df_sim_array = []
    for df in single_day:
        df_sim_array.append(df.rename(columns={'LONN.SW': 'LONN.SE', 'SIKA.SW': 'SIKA.SE'}))
    
    prices = pricing_multiple_no_autocall(df_sim_array)
    est_price.append(sum(prices) / len(prices))
print(est_price)

[969.5923690030505, 984.2207743757805, 990.5630487743467, 976.7104002070903, 983.3203000643206, 997.4387062352808, 953.3594091107491, 941.2641047242605, 955.0786620862184, 967.988844804869, 951.6714752431169, 979.2327565699785, 959.6615388555587, 983.4540833305799, 982.2533418459518, 998.9118568264424, 947.8825239682592, 977.7680382652934, 990.5174633247065, 1001.4301585772184, 1003.577324948565, 983.0673711805233, 972.5246071287688, 1003.7177361468458, 1007.470481047923, 1004.242333228093, 1032.602144329112, 1025.7804191952296, 870.894246091833, 851.6137890848676, 871.8661660120986, 876.2552096008528, 892.2999393555802, 879.8743884947603, 867.4519629983745, 877.6027639339103, 867.3315891736555, 919.5851004955414, 865.8988236915678, 868.3580319980471, 860.6883811709507, 863.1378694355285, 837.2059877653281, 913.5938985868848, 905.3409063205409, 877.9689802613257, 923.3390675328736, 895.812815016821, 870.9030458473417, 727.544221973209, 717.9798110243869, 724.0534226662111, 743.09086878

In [ ]:
plt.plot(df_product)
plt.plot(df_upper_bound)